In [1]:
!pip install keras-layer-normalization

  Preparing metadata (setup.py) ... done
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4654 sha256=958986dc0b70c125ccb1a18b16c1370a67eef5ccd744f01add6e698a68b09a3c
  Stored in directory: /root/.cache/pip/wheels/ed/3a/4b/21db23c0cc56c4b219616e181f258eb7c57d36cc5d056fae9a
Successfully built keras-layer-normalization


In [9]:
import os
import imageio
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from skimage.transform import resize
import tensorflow.keras.backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image,draw_boxes, scale_boxes
from keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body
%matplotlib inline


In [10]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    #Computing box scores
    box_scores = box_confidence*box_class_probs
    #Finding the index of the class with maximum box score
    box_classes = K.argmax(box_scores, -1)
    #Getting the corresponding box score
    box_class_scores = K.max(box_scores,-1)
    #Creating a filtering mask. The mask will be true for all the boxes        we intend to keep (pc >= threshold) and false for the rest
    filtering_mask = box_class_scores>threshold
    #Applying the mask to scores, boxes and classes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes,filtering_mask)
    return scores, boxes, classes


In [11]:
def iou(box1, box2):
    #Calculating (xi1,yi1,xi2,yi2) of the intersection of box1 and box2
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    #Calculating the area of intersection
    inter_area = (yi2-yi1)*(xi2-xi1)
    #Calculating the areas of box1 and box2 using the same formula
    box1_area = (box1[3] - box1[1])*(box1[2] - box1[0])
    box2_area = (box2[3] - box2[1])*(box2[2] - box2[0])
    #Calculating the union area by using the formula: union(A,B) = A+B-Inter(A,B)
    union_area = box1_area + box2_area - inter_area
    #Calculating iou
    iou = inter_area/union_area

    return iou


In [12]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    #tensor used in tf.image.non_max_suppression()of size 'max_boxes'
    max_boxes_tensor = K.variable(max_boxes, dtype = 'int32')
    #initiating the tensor
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    #Using the tensorflow function tf.image.non_max_suppression to get the indices of boxes kept
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes, iou_threshold)
    #Using K.gather to individually access scores, boxes and classes from nms_indices
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)

    return scores, boxes, classes

In [13]:
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes = 10, score_threshold = .6, iou_threshold = .5):
    '''
    yolo_outputs contains:
    box_confidence, box_xy, box_wh, box_class_probs
    '''
    #Retrieving output
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
    #Converting the boxes for filtering functions
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    #Using the function defined before to remove boxes with less confidence score
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = score_threshold)
    #Scaling the boxes
    boxes = scale_boxes(boxes, image_shape)
    #Using the function defined before for non-max suppression
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)

    return scores, boxes, classes


In [17]:
import tensorflow.python.keras.backend as K
sess = K.get_session()
class_names = read_classes("coco_classes.txt")
anchors = read_anchors("yolo_anchors.txt")
yolo_model = load_model("yolo.h5")
#yolo_model=tf.keras.saving.load_model("yolo.h5")



OSError: ignored

In [ ]:
#Converting the output of model into usable bounding box tensors
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))
#Filtering the boxes
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)


NameError: ignored

In [16]:
def predict(sess, image_file):
    #Preprocessing the image
    image, image_data = preprocess_image("images/"+image_file, model_image_size = (608,608))
    #Running the session and feeding the input to it
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes],feed_dict = {yolo_model.input: image_data, K.learning_phase(): 0})
    #Prints the predicted information
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    #Generates color for drawing bounding boxes
    colors = generate_colors(class_names)
    #Draws bounding boxes on the image file
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    #Saving the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality = 150)
    #Displaying the results in notebook
    output_image = imageio.imread(os.path.join("out", image_file))
    plt.figure(figsize=(12,12))
    imshow(output_image)
    return out_scores, out_boxes, out_classes
